In [1]:
'''
import numpy   as np 
import scipy.misc # to visualize only  

#x = np.loadtxt("data/train_x.csv", delimiter=",") # load from text 
#y = np.loadtxt("data/train_y.csv", delimiter=",") 
#x = x.reshape(-1, 64, 64) # reshape 
#y = y.reshape(-1, 1)
np.save("data/"+"X_train",x)
np.save("data/"+"y_train",y)
print("X.shape",x.shape, "Y.shape",y.shape)
'''

'\nimport numpy   as np \nimport scipy.misc # to visualize only  \n\n#x = np.loadtxt("data/train_x.csv", delimiter=",") # load from text \n#y = np.loadtxt("data/train_y.csv", delimiter=",") \n#x = x.reshape(-1, 64, 64) # reshape \n#y = y.reshape(-1, 1)\nnp.save("data/"+"X_train",x)\nnp.save("data/"+"y_train",y)\nprint("X.shape",x.shape, "Y.shape",y.shape)\n'

In [2]:
import numpy as np
x = np.load("data/X_train.npy")
y = np.load("data/y_train.npy")

IOError: [Errno 2] No such file or directory: 'data/X_train.npy'

In [2]:
#preprocessing
#using 1 hot encoding encode the 40 unique classes of y 
decode = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 24, 25, 27, 28, 30, 32, 35, 36, 40, 42, 45, 48, 49, 54, 56, 63, 64, 72, 81]
encodedY = np.zeros((50000,40))
for i in range(0,49999):
    index = decode.index(y[i])
    encodedY[i][index] = 1

flatX = np.zeros((50000,4096))
for i in range(0,49999):
    flatX[i] = x[i].flatten()
    #normalize input data
    mean = np.mean(flatX[i])
    flatX[i] = (flatX[i]- mean)/128.0
    
#for early testing, use 8% as training set and 2% as testing set
inputX = flatX[0:800]
validX = flatX[800:1000]
inputY = encodedY[0:800]
validY = encodedY[800:1000]

In [7]:
from numpy import exp, array, random, dot
from sklearn.utils import shuffle

class FeedForwardNeuralNetwork():
    #Nlayer must be >=2
    def __init__(self,NLayer,Nneuron,ranSeed,Ninput,Noutput,bias):
        #specify the number of hidden layers , number of neurons needed per layer, number of weights per neuron
        #ranSeed specify the random seed used in initializing weights
        #activation specified what activation function is to be used, pass in 0 to use default sigmoid function
        #-------------------------------
        np.random.seed(ranSeed)
        self.bias = bias
        self.Nneuron = Nneuron
        self.Ninput = Ninput
        self.Noutput = Noutput
        self.NLayer = NLayer
        #input weights
        self.InputWeight = np.random.uniform(0,1,(Nneuron,Ninput))
        #this is weight for neurons from hidden layer 1 to (n-1)
        self.neurons = np.random.uniform(0,1,(NLayer-1,Nneuron,Nneuron))
        #store output of each layer
        self.O = np.random.uniform(0,1,(NLayer,Nneuron))
        #the weight for the last layer of neurons
        self.OutputWeight = np.random.uniform(0,1,(Nneuron,Noutput))
        #encoding output layer with multi-class domains, each output neuron represents a discrete output value
        #see as an classification task
        self.output = np.random.uniform(0,1,Noutput)
    
    #allows users to manually set all the weights
    def set_Weights(self,InputWeights,OutputWeights,HiddenWeights):
        self.OutputWeight = OutputWeights
        self.InputWeight = InputWeights
        self.neurons = HiddenWeights
    
    
    
    #default activation function, normalize input into 0 and 1
    # normalise them between 0 and 1.
    def sigmoid(self, x):
        return 1 / (1 + exp(-x))

    # The derivative of the Sigmoid function.
    # This is the gradient of the Sigmoid curve.
    def sigmoid_derivative(self, x):
        return x * (1 - x)

    #train the neural network
    #can change learning rate
    def train(self, num_training_example, training_inputs, training_outputs, num_iteration, learning_rate):
        #use to store outputs
        outputArr = np.zeros((num_training_example,self.Noutput))
        for iteration in xrange(num_iteration):
            for i in range(0,num_training_example):
                #---------------------------------------------
                #need a output layer special gamma
                #-------F--------------------------------------
                gammaOutput = np.zeros(self.Noutput)
                # gamma is the error correction for all neurons in NN
                # gamma values are used to backpropagate
                gamma = np.zeros((self.NLayer,self.Nneuron))
                #---------------------------------------------
                # Forward pass
                self.forward(training_inputs[i])
                outputArr[i] = np.copy(self.output)
                print("forward pass result add =",np.sum(self.output))
                #---------------------------------------------
                #compute corrections
                # first calculate correction in the output layer
                for k in range(0,self.Noutput):
                    gammaOutput[k] = self.sigmoid_derivative(self.output[k])*(training_outputs[i][k]-self.output[k])
                    #calculate the gamma values for the last hidden layer
                    #And update weights for output layer
                    for z in range(0,self.Nneuron):
                        gamma[self.NLayer-1][z] = self.sigmoid_derivative(self.O[self.NLayer-1][z])*dot(gammaOutput,self.OutputWeight[z])
                for z in range(0,self.Nneuron):
                    for k in range(0,self.Noutput):
                        #print ("position is " + str(z) + str(k))
                        #print (self.OutputWeight[z][k])
                        self.OutputWeight[z][k] = self.OutputWeight[z][k] + (learning_rate*gammaOutput[k]*
                                                                              self.O[self.NLayer-1][z])
                        #print (self.OutputWeight[z][k])
                        #print ((learning_rate*gammaOutput[k]*self.O[self.NLayer-1][z]))
                #--------------------------------------
                #calculate the backprop for hidden layer
                #j is counting the current layer
                #z is counting the neuron position
                #propagate gamma values
                for j in range(self.NLayer-2,-1,-1):
                    for z in range(0,self.Nneuron):
                        #multiple the derivative with all the connected neuron's gamma                
                        #SumGamma will compute all the propagated gammas
                        gamma[j][z] = self.sigmoid_derivative(self.O[j][z])*(self.SumGamma(gamma,j,z))
                        #update the weights now
                        for k in range(0,self.Nneuron):
                            self.neurons[j][z][k] = self.neurons[j][z][k] + (learning_rate*gamma[j][z]*
                            self.O[j][k])
                #---------------------------------------
        print (outputArr)
                
        
    #compute the propagated gamma of a neuron
    def SumGamma(self,gamma,Layer,position):
        output = dot(gamma[Layer+1],self.neurons[Layer][position])
        return output
                
    # pass input through all the layers until the output layer
    def forward(self, inputs):
        #first pass from input layer to hidden layer
        current = np.zeros(self.Nneuron)
        for i in range(0,self.Nneuron):
            current[i] = self.sigmoid(dot(inputs,self.InputWeight[i])+self.bias)
            self.O[0][i] = current[i]
        #passing through hidden layers
        for i in range(0,self.NLayer-1):
            for j in range(0,self.Nneuron):
                current[j] = self.sigmoid(dot(self.O[i],self.neurons[i][j])+self.bias)
            self.O[i+1] = np.copy(current)
        #now from hidden layer to output layer
        #the weight for the last hidden layer is the weight for the output layer 
        #---------------------------------------------------------------
        for i in range(0,self.Noutput):
            self.output[i]= self.sigmoid(dot(current,self.OutputWeight[:,i])+self.bias)
        #----------------------------------------------------------------
        
    #a simple function to see the validation accuracy    
    def validate(self, num_validate_example, validating_inputs, validating_outputs):
        count = 0
        for i in range(0,num_validate_example):
            self.forward(validating_inputs[i])
            if(self.output.index(1) == validating_outputs[i].index(1)):
                count = count + 1
        print("accuracy is" + str(count*1.0/num_validate_example))
    
        
        
if __name__ == "__main__":

    #Intialise a single neuron neural network.
    #def __init__(self,NLayer,Nneuron,ranSeed,Ninput,Noutput,bias):
    #neural_network = FeedForwardNeuralNetwork(3,200,1,4096,40,1)
    neural_network = FeedForwardNeuralNetwork(2,3,1,8,8,1)
    #neural_network.set_Weights(InputWeights,OutputWeights,HiddenWeights)
    trainX = [[1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,1],[0,1,0,0,0,0,0,0],[0,0,1,0,0,0,0,0],[0,0,0,1,0,0,0,0],[0,0,0,0,1,0,0,0],[0,0,0,0,0,1,0,0],[0,0,0,0,0,0,1,0]]
    trainY = [[1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,1],[0,1,0,0,0,0,0,0],[0,0,1,0,0,0,0,0],[0,0,0,1,0,0,0,0],[0,0,0,0,1,0,0,0],[0,0,0,0,0,1,0,0],[0,0,0,0,0,0,1,0]]
    trainX,trainY = shuffle(trainX,trainY)
    neural_network.train(8,trainX,trainY,10000,0.01)
    #def train(self, num_training_example, training_inputs, training_outputs, num_iteration, learning_rate):
    #neural_network.train(800,inputX,inputY,20,0.001)
    #def validate(self, num_validate_example, validating_inputs, validating_outputs):
    #neural_network.validate(200,validX,validY)
    


 ('forward pass result add =', 7.1290870439181866)
('forward pass result add =', 7.1242309849714651)
('forward pass result add =', 7.1257573088390096)
('forward pass result add =', 7.1167415400967258)
('forward pass result add =', 7.1236904416476978)
('forward pass result add =', 7.1181395185409455)
('forward pass result add =', 7.1132014495042482)
('forward pass result add =', 7.1157941708037846)
('forward pass result add =', 7.1171703991023021)
('forward pass result add =', 7.11228483699756)
('forward pass result add =', 7.1137773607737103)
('forward pass result add =', 7.1047593167077636)
('forward pass result add =', 7.1116245389619994)
('forward pass result add =', 7.1060993180793783)
('forward pass result add =', 7.1011475871128162)
('forward pass result add =', 7.1036875121218266)
('forward pass result add =', 7.105012306352652)
('forward pass result add =', 7.1000972920513838)
('forward pass result add =', 7.1015548000648945)
('forward pass result add =', 7.0925361520269288)
('

('forward pass result add =', 6.1775058654398318)
('forward pass result add =', 6.1741474462097594)
('forward pass result add =', 6.1686760846865116)
('forward pass result add =', 6.1670017756167894)
('forward pass result add =', 6.1615427781735601)
('forward pass result add =', 6.1584149047213348)
('forward pass result add =', 6.1575682809660819)
('forward pass result add =', 6.1536084230656414)
('forward pass result add =', 6.1508077159984129)
('forward pass result add =', 6.1473159515288982)
('forward pass result add =', 6.1418717779349628)
('forward pass result add =', 6.1401123928784376)
('forward pass result add =', 6.1348300885624649)
('forward pass result add =', 6.1313797371371539)
('forward pass result add =', 6.1307115453494898)
('forward pass result add =', 6.1268234443962166)
('forward pass result add =', 6.1238656287501643)
('forward pass result add =', 6.1202401515067493)
('forward pass result add =', 6.114825579959458)
('forward pass result add =', 6.1129812232459475)
(

('forward pass result add =', 5.153479808746436)
('forward pass result add =', 5.149326669929577)
('forward pass result add =', 5.1519567009061698)
('forward pass result add =', 5.1372463113526656)
('forward pass result add =', 5.1437588890489199)
('forward pass result add =', 5.1432944100963862)
('forward pass result add =', 5.1351552570693997)
('forward pass result add =', 5.1273700236625874)
('forward pass result add =', 5.1242696145199567)
('forward pass result add =', 5.1200681084431041)
('forward pass result add =', 5.1229514549748734)
('forward pass result add =', 5.1079406660780755)
('forward pass result add =', 5.1146661635617274)
('forward pass result add =', 5.1143123073803052)
('forward pass result add =', 5.1060400647131523)
('forward pass result add =', 5.0981533244356525)
('forward pass result add =', 5.0951456632385526)
('forward pass result add =', 5.0908969463537996)
('forward pass result add =', 5.0940327572222568)
('forward pass result add =', 5.0787244840528976)
('

('forward pass result add =', 2.7058882108038107)
('forward pass result add =', 2.7277766038517051)
('forward pass result add =', 2.736207149643807)
('forward pass result add =', 2.7183076770679993)
('forward pass result add =', 2.7033194397347513)
('forward pass result add =', 2.7091948033193765)
('forward pass result add =', 2.7023139906004743)
('forward pass result add =', 2.7249215315037922)
('forward pass result add =', 2.6883741803267975)
('forward pass result add =', 2.7103621357337104)
('forward pass result add =', 2.7188710357616888)
('forward pass result add =', 2.7009386899724612)
('forward pass result add =', 2.685936198505543)
('forward pass result add =', 2.6918836688918177)
('forward pass result add =', 2.6849960380879501)
('forward pass result add =', 2.7077320542046204)
('forward pass result add =', 2.6710919354639842)
('forward pass result add =', 2.6931771238430842)
('forward pass result add =', 2.7017631691922155)
('forward pass result add =', 2.6837996650057869)
('

('forward pass result add =', 1.69135901394545)
('forward pass result add =', 1.7029643426300516)
('forward pass result add =', 1.6855465645549546)
('forward pass result add =', 1.6717329548608659)
('forward pass result add =', 1.680299614162974)
('forward pass result add =', 1.6733440115916469)
('forward pass result add =', 1.6999045251746812)
('forward pass result add =', 1.6630492452709942)
('forward pass result add =', 1.6873894501867903)
('forward pass result add =', 1.6989969839593977)
('forward pass result add =', 1.6815910009054491)
('forward pass result add =', 1.6677895464747672)
('forward pass result add =', 1.676357859545198)
('forward pass result add =', 1.6694041419991272)
('forward pass result add =', 1.6959594910726574)
('forward pass result add =', 1.6591265667294293)
('forward pass result add =', 1.6834570437707348)
('forward pass result add =', 1.6950666633261986)
('forward pass result add =', 1.6776724442954303)
('forward pass result add =', 1.6638830890347811)
('fo

...

('forward pass result add =', 1.4243958247767472)
('forward pass result add =', 1.4359188728941081)
('forward pass result add =', 1.4194796273535086)
('forward pass result add =', 1.4066172113521418)
('forward pass result add =', 1.4151211690486778)
('forward pass result add =', 1.4083644297342599)
('forward pass result add =', 1.4341441164460491)
('forward pass result add =', 1.3992526889990304)
('forward pass result add =', 1.4225437712043723)
('forward pass result add =', 1.4340645617207568)
('forward pass result add =', 1.4176334690563306)
('forward pass result add =', 1.4047786312701924)
('forward pass result add =', 1.413280901642096)
('forward pass result add =', 1.4065261146429453)
('forward pass result add =', 1.4322972230256594)
('forward pass result add =', 1.3974227862223856)
('forward pass result add =', 1.4207037268516791)
('forward pass result add =', 1.4322222506990003)
('forward pass result add =', 1.4157992774891404)
('forward pass result add =', 1.402951982674242)
('

('forward pass result add =', 1.3454302392237829)
('forward pass result add =', 1.3328829136190927)
('forward pass result add =', 1.3413053452274182)
('forward pass result add =', 1.3346339743679316)
('forward pass result add =', 1.3600341679026435)
('forward pass result add =', 1.3258622023174178)
('forward pass result add =', 1.3487147400463244)
('forward pass result add =', 1.360125886982543)
('forward pass result add =', 1.3440360068842929)
('forward pass result add =', 1.3314948358003558)
('forward pass result add =', 1.3399154582775004)
('forward pass result add =', 1.3332458375465097)
('forward pass result add =', 1.3586381832015153)
('forward pass result add =', 1.3244805187790607)
('forward pass result add =', 1.3473241709172588)
('forward pass result add =', 1.3587328818604776)
('forward pass result add =', 1.3426497365836001)
('forward pass result add =', 1.3301146931217056)
('forward pass result add =', 1.3385335066844388)
('forward pass result add =', 1.3318656314692707)
(

('forward pass result add =', 1.2357947520689889)
('forward pass result add =', 1.220352467875911)
('forward pass result add =', 1.2083912114652477)
('forward pass result add =', 1.2166102138880142)
('forward pass result add =', 1.2101184586499989)
('forward pass result add =', 1.2347087131136285)
('forward pass result add =', 1.2019341854478029)
('forward pass result add =', 1.2238956936354393)
('forward pass result add =', 1.2350323437484119)
('forward pass result add =', 1.2195943288489208)
('forward pass result add =', 1.2076368397231894)
('forward pass result add =', 1.2158543516416183)
('forward pass result add =', 1.2093638312027268)
('forward pass result add =', 1.2339485123069875)
('forward pass result add =', 1.2011831755082238)
('forward pass result add =', 1.2231386980076633)
('forward pass result add =', 1.2342733337829817)
('forward pass result add =', 1.2188395728652819)
('forward pass result add =', 1.2068858368584676)
('forward pass result add =', 1.2151018617893681)
(

('forward pass result add =', 1.1662423883147497)
('forward pass result add =', 1.154556779216847)
('forward pass result add =', 1.1626614084378903)
('forward pass result add =', 1.1562625252941818)
('forward pass result add =', 1.1804343664951262)
('forward pass result add =', 1.1483383050865712)
('forward pass result add =', 1.1698546491858561)
('forward pass result add =', 1.180836667060722)
('forward pass result add =', 1.1657118561114408)
('forward pass result add =', 1.1540290184534334)
('forward pass result add =', 1.162132445262497)
('forward pass result add =', 1.1557345214383319)
('forward pass result add =', 1.1799020512306515)
('forward pass result add =', 1.1478128700805186)
('forward pass result add =', 1.1693246684961491)
('forward pass result add =', 1.1803050602825322)
('forward pass result add =', 1.1651834179070959)
('forward pass result add =', 1.1535033420952741)
('forward pass result add =', 1.1616055696578591)
('forward pass result add =', 1.1552086023314625)
('f

KeyboardInterrupt: 